In [1]:
def make_hicc_json(input_files, env, output_bucket, accession, ncores):
    input_json = {'input_files': input_files,
                  'output_bucket': output_bucket,
                  'workflow_uuid': "1348354f-49e5-4c33-afab-9ec90d65faf3",
                  "app_name": "hi-c-processing-partc/2",
                  "parameters": {
                      "ncores" :  ncores
                  },
                  "_tibanna": {"env": env, "run_type": "hic_part_c", "run_id": accession}
                  }
    return input_json

def make_input_file_json(obj_ids, arg_name, tibanna, bucket):
    '''
    obj_ids can be either a string or a list.
    {
      "bucket_name": "%s",
      "object_key": "%s",
      "uuid" : "%s",
      "workflow_argument_name": "%s"
    }
    '''
    ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
    
    if not isinstance(obj_ids, list):
        obj_ids = [ obj_ids ]
        
    object_key_list = []
    uuid_list = []
    
    for obj_id in obj_ids:
        metadata = ff_utils.get_metadata(obj_id, connection=ff)
         
        # just make sure the file is on s3, otherwise bail
        print("looking for upload key %s, on bucket %s" % 
              (metadata['upload_key'],
               bucket))
        if tibanna.s3.does_key_exist(metadata['upload_key'], bucket=bucket):
            object_key_list.append(metadata['upload_key'].split('/')[1])
            uuid_list.append(metadata['uuid'])
            
    if len(uuid_list)==1:
        uuid_list = uuid_list[0]
    if len(object_key_list)==1:
        object_key_list = object_key_list[0]
        
    data = {'bucket_name' : bucket,
            'object_key' :  object_key_list,
            'uuid' : uuid_list,
            'workflow_argument_name': arg_name
            }
    return data

In [6]:
from core.utils import Tibanna
from core.utils import run_workflow
from core import ff_utils
import time

# testportal
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)

#rao13 cool file 4DNFIT41X0MQ
#jin   cool file 4DNFIIIHUUHH
all_cools = [
['/files-processed/4DNFIIIHUUHH/',"Jin_et_al"]
]


output_file_bucket = tibanna.s3.outfile_bucket
raw_file_bucket = tibanna.s3.raw_file_bucket

for cool_accession, set_name in all_cools:
    cool_file= make_input_file_json(cool_accession, 'input_cool', tibanna, output_file_bucket)
    ncores = 8

    input_files = [cool_file]
    if all(input_files):
        name = set_name + '_' + cool_accession.split('/')[2]
        input_json = make_hicc_json(input_files, env, output_file_bucket, name, ncores)
        print input_json
        res = run_workflow(input_json)
    else:
        print("some files not found on s3.  Investigate this list %s" % input_files)
    print('Done')

looking for upload key 439cd2d1-9586-4942-b3cd-8b0ea5971206/4DNFIIIHUUHH.cool, on bucket elasticbeanstalk-fourfront-webdev-wfoutput
{'parameters': {'ncores': 8}, '_tibanna': {'run_type': 'hic_part_c', 'env': 'fourfront-webdev', 'run_id': 'Jin_et_al_4DNFIIIHUUHH'}, 'output_bucket': 'elasticbeanstalk-fourfront-webdev-wfoutput', 'input_files': [{'workflow_argument_name': 'input_cool', 'bucket_name': 'elasticbeanstalk-fourfront-webdev-wfoutput', 'uuid': u'439cd2d1-9586-4942-b3cd-8b0ea5971206', 'object_key': u'4DNFIIIHUUHH.cool'}], 'workflow_uuid': '1348354f-49e5-4c33-afab-9ec90d65faf3', 'app_name': 'hi-c-processing-partc/2'}
about to start run hic_part_c_Jin_et_al_4DNFIIIHUUHH
response from aws was: 
 {u'startDate': datetime.datetime(2017, 7, 18, 16, 4, 44, 452000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '579d2d2c-6bf4-11e7-8dd3-7d1021684ac9', 'HTTPHeaders': {'x-amzn-requestid': '579d2d2c-6bf4-11e7-8dd3-7d1021684ac9', 'content-length'